## Installation

In [ ]:
pip install wandb numpy pandas matplotlib torch torchvision

## Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip /content/drive/MyDrive/DL_Assignment2/Dataset/nature_12K.zip -d /content/drive/MyDrive/DL_Assignment2/Dataset

In [20]:
import os
os.chdir("/content/drive/MyDrive/DL_Assignment2/partB")

## Libraries

In [34]:
%%writefile libraries.py
import torch
import os
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from torch.utils.data import Subset, DataLoader
import numpy as np
import torch.nn as nn
import torch.optim as optim
import wandb
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
import gc
from PIL import Image

Writing libraries.py


## Dataset loader

In [35]:
%%writefile data_loader.py
import torch
from torch.utils.data import Subset, DataLoader
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import os
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from PIL import Image
import numpy as np

def validationDataSplit(train_dataset):
  classLabels = [label for _,label in train_dataset.samples]
  num_classes = len(np.unique(classLabels))

  sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
  train_indices, val_indices = next(sss.split(train_dataset.samples, classLabels))

  train_subset = Subset(train_dataset, train_indices)
  val_subset = Subset(train_dataset, val_indices)
  return train_subset, val_subset, num_classes


def load_data(base_dir, isDataAug, batch_size):
  train_dir = os.path.join(base_dir, 'train')
  test_dir = os.path.join(base_dir, 'val')

  train_transform, test_transform = None, None

  # model_transforms = EfficientNet_V2_S_Weights.IMAGENET1K_V1.transforms()
  # print(model_transforms)

  if isDataAug == False:
    train_transform = transforms.Compose([
      transforms.Resize(384, interpolation=Image.BILINEAR),
      transforms.CenterCrop(384),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
  else:
    train_transform = transforms.Compose([
      transforms.Resize(384, interpolation=Image.BILINEAR),
      transforms.CenterCrop(384),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(10),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

  test_transform = transforms.Compose([
      transforms.Resize(384, interpolation=Image.BILINEAR),
      transforms.CenterCrop(384),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

  train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
  test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
  train_dataset, val_dataset, num_classes = validationDataSplit(train_dataset)

  # print(f"inp: {train_dataset[0][0].shape} {train_dataset[0][1]}")

  train_loader = DataLoader(train_dataset,shuffle=True,num_workers=2,batch_size=batch_size,pin_memory=True)
  test_loader = DataLoader(test_dataset,shuffle=True,num_workers=2,batch_size=64,pin_memory=True)
  val_loader = DataLoader(val_dataset,shuffle=True,num_workers=2,batch_size=64,pin_memory=True)

  return train_loader, test_loader, val_loader, num_classes

Overwriting data_loader.py


## Training CNN

In [36]:
%%writefile neural_network.py
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights

class ConvolutionalNeuralNetwork(nn.Module):
  activationFunctionsMap = {"ReLU": nn.ReLU, "GELU": nn.GELU, "SiLU": nn.SiLU}
  # optimizersMap = {"sgd": optim.SGD, "rmsprop": optim.RMSprop, "adam": optim.Adam}

  def __init__(self, num_classes,
               num_filters, filter_sizes,
               activationFun, optimizer,
               n_neurons_denseLayer,
               isBatchNormalization, dropout,
               learning_rate=0.001,
               momentum=0.5, beta = 0.9,
               beta1=0.9, beta2=0.99,
               epsilon=1e-8, weight_decay=0.0001):
    super(ConvolutionalNeuralNetwork, self).__init__()
    self.num_classes = num_classes
    self.num_filters = num_filters
    self.filter_sizes = filter_sizes
    self.activationFun = ConvolutionalNeuralNetwork.activationFunctionsMap[activationFun]
    # self.optimizer = ConvolutionalNeuralNetwork.optimizersMap[optimizer]

    self.n_neurons_denseLayer = n_neurons_denseLayer
    self.isBatchNormalization = isBatchNormalization
    self.dropout = dropout

    self.lr = learning_rate
    self.momentum = momentum
    self.betas = (beta1, beta2)
    self.eps = epsilon
    self.alpha = beta
    self.weight_decay = weight_decay
    # self.count = 0
    # self.cached_inputs = None    # dataloader shuffling within batches to cache (else needs to make shuffle = False)

    self.defineModel()

    trainable_parameters = [p for p in self.parameters() if p.requires_grad == True]

    if(optimizer == "sgd"):
      self.optimizer = optim.SGD(trainable_parameters, lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
    elif(optimizer == "rmsprop"):
      self.optimizer = optim.RMSprop(trainable_parameters, lr=self.lr, alpha=self.alpha, eps=self.eps, weight_decay=self.weight_decay)
    elif(optimizer == "adam"):
      self.optimizer = optim.Adam(trainable_parameters, lr=self.lr, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay)
    # print(trainable_parameters)

  def defineModel(self):
    self.model = efficientnet_v2_s(weights="IMAGENET1K_V1")

    # Freezing all layers except the last layer
    for param in self.model.parameters():
        param.requires_grad = False

    # Replacing last layer (its a classifier at the end containing dropout followed by linear layer)
    num_in_features_last_layer = self.model.classifier[1].in_features
    self.model.classifier[1] = nn.Linear(num_in_features_last_layer, self.num_classes)

    # last layer is trainable
    for name, param in self.model.classifier[1].named_parameters():
        param.requires_grad = True

  def forward(self, inputs):
    return self.model(inputs)

  def backward(self, outputs, labels):
    loss = nn.CrossEntropyLoss()(outputs, labels)
    loss.backward()

  def updateWeights(self):
    self.optimizer.step()

Overwriting neural_network.py


## Accuracy Calculation

In [37]:
%%writefile accuracy_calculation.py
import torch
import torch.nn as nn
from torch.utils.data import Subset, DataLoader


def findOutputs(device, cnn, inputDataLoader, isTestData=False):
  cnn.eval()  # setting the model to evaluation model
  outputs = []
  total_loss = 0.0
  n_correct = 0
  n_correct_top5 = 0
  n_correct_top2 = 0
  n_samples = 0

  with torch.no_grad():
    for batch_idx, (x_batch, y_batch) in enumerate(inputDataLoader):
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)
      batch_outputs = cnn(x_batch)

      loss = nn.CrossEntropyLoss()(batch_outputs, y_batch)
      total_loss += loss.item() * x_batch.size(0)

      y_pred_batch = torch.argmax(batch_outputs, dim=1)
      n_correct += (y_pred_batch == y_batch).sum().item()
      n_samples += x_batch.size(0)

      if isTestData == True:
          y_pred_batch_top5 = torch.topk(batch_outputs, 5, dim=1).indices
          n_correct_top5 += y_pred_batch_top5.eq(y_batch.view(-1, 1)).sum().item()

          y_pred_batch_top2 = torch.topk(batch_outputs, 2, dim=1).indices
          n_correct_top2 += y_pred_batch_top2.eq(y_batch.view(-1, 1)).sum().item()
      outputs.append(batch_outputs)

  outputs = torch.cat(outputs)
  accuracy = (n_correct * 100.0) / n_samples
  avg_loss = total_loss / n_samples

  top5_accuracy = None
  top2_accuracy = None
  if isTestData == True:
      top5_accuracy = (n_correct_top5 * 100.0) / n_samples
      top2_accuracy = (n_correct_top2 * 100.0) / n_samples
  return outputs, accuracy, avg_loss, top5_accuracy, top2_accuracy

Overwriting accuracy_calculation.py


## Training (ArgParser included)

In [40]:
%%writefile train_local.py
import os
import gc
import wandb
import torch
from neural_network import *
from data_loader import *
from accuracy_calculation import *
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def trainNeuralNetwork_local(args):
  wandb.login()
  wandb.init(mode="online")
  wandb.init(project=args.wandb_project, entity=args.wandb_entity)
  if args.isDataAug == "True":
    isDataAug = True
  else:
    isDataAug = False

  train_loader, test_loader, val_loader, num_classes = load_data(args.base_dir, isDataAug, args.batch_size)
  activationFun = "SiLU"    # have no effect since using pre-trained model
  optimizer = args.optimizer
  learning_rate = args.learning_rate
  momentum = args.momentum
  beta = args.beta
  beta1 = args.beta1
  beta2 = args.beta2
  epsilon = args.epsilon
  weight_decay = args.weight_decay
  dropout = 0   # have no effect since using pre-trained model
  num_filters = [32, 32,32,32,32]   # have no effect since using pre-trained model
  filter_sizes = [3,3,3,3,3]   # have no effect since using pre-trained model
  n_neurons_denseLayer = 128  # have no effect since using pre-trained model
  isBatchNormalization = False  # have no effect since using pre-trained model

  wandb.run.name = f"{activationFun}_{optimizer}_{dropout}_{n_neurons_denseLayer}_DataAug-{isDataAug}_BatchNorm-{isBatchNormalization}"
  best_val_accuracy = 0.0
  best_accuracy_epoch = -1

  cnn = ConvolutionalNeuralNetwork(num_classes,
                                   num_filters, filter_sizes,
                                   activationFun, optimizer,
                                   n_neurons_denseLayer,
                                   isBatchNormalization, dropout,
                                   learning_rate,
                                   momentum, beta,
                                   beta1, beta2,
                                   epsilon, weight_decay)
  cnn.to(device)

  epochs = args.epochs
  for epochNum in range(epochs):
    print(f"Epoch {epochNum}:")
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
      if(batch_idx % 40 == 0):
        print(f"Batch idx {batch_idx} running")
        # break
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)
      cnn.optimizer.zero_grad()
      outputs = cnn(x_batch)
      cnn.backward(outputs, y_batch)
      cnn.updateWeights()
      del x_batch, y_batch, outputs

    # Validation accuracy
    val_outputs, val_accuracy, val_loss, _, _ = findOutputs(device, cnn, val_loader)
    print(f"validation: loss={val_loss}, accuracy={val_accuracy}")

    # Train accuracy
    train_outputs, train_accuracy, train_loss, _, _ = findOutputs(device, cnn, train_loader)
    print(f"training: loss={train_loss}, accuracy={train_accuracy}")

    if val_accuracy > best_val_accuracy:
      best_val_accuracy = val_accuracy
      best_accuracy_epoch = epochNum

    wandb.log({
        "epoch": epochNum + 1,
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy
        },commit=True)
    del val_outputs, train_outputs
    gc.collect()
    torch.cuda.empty_cache()

  wandb.log({
      "best_acc_epoch": best_accuracy_epoch,
      "best_val_accuracy": best_val_accuracy
  })

  test_outputs, test_accuracy, test_loss, test_top5_accuracy, test_top2_accuracy = findOutputs(device, cnn, test_loader, True)
  print(f"testing: loss={test_loss}, top1_accuracy={test_accuracy}, top5_accuracy = {test_top5_accuracy}, top2_accuracy = {test_top2_accuracy}")

  wandb.log({
      "test_loss": test_loss,
      "test_top1_accuracy": test_accuracy,
      "test_top5_accuracy": test_top5_accuracy,
      "test_top2_accuracy": test_top2_accuracy
  })
  del cnn,train_loader, test_loader, val_loader
  gc.collect()
  torch.cuda.empty_cache()

  wandb.finish()

Overwriting train_local.py


### ArgParser

In [41]:
%%writefile argument_parser.py
import argparse

def parse_arguments():
    parser = argparse.ArgumentParser()

    parser.add_argument("-wp", "--wandb_project", type=str, default="DA6401_Assignment2",
                        help="Project name used to track experiments in Weights & Biases dashboard")
    parser.add_argument("-we", "--wandb_entity", type=str, default="nikhithaa-iit-madras",
                        help="Wandb Entity used to track experiments in the Weights & Biases dashboard.")
    parser.add_argument("-bd", "--base_dir", type=str, default="inaturalist_12K",
                        help="Base directory where dataset (train/val folders) are present")
    parser.add_argument("-e", "--epochs", type=int, default=10,
                        help="Number of epochs to train neural network")
    parser.add_argument("-b", "--batch_size", type=int, default=32,
                        help="Batch size used to train neural network")
    parser.add_argument("-o", "--optimizer", type=str, choices=["sgd", "rmsprop", "adam"], default="sgd",
                        help="Choose one among these optimizers: ['sgd', 'rmsprop', 'adam']")
    parser.add_argument("-lr", "--learning_rate", type=float, default=0.001,
                        help="Learning rate used to optimize model parameters")
    parser.add_argument("-m", "--momentum", type=float, default=0.9,
                        help="Momentum used by momentum and nag optimizers")
    parser.add_argument("-beta", "--beta", type=float, default=0.9,
                        help="Beta used by rmsprop optimizer")
    parser.add_argument("-beta1", "--beta1", type=float, default=0.9,
                        help="Beta1 used by adam and nadam optimizers")
    parser.add_argument("-beta2", "--beta2", type=float, default=0.999,
                        help="Beta2 used by adam and nadam optimizers")
    parser.add_argument("-eps", "--epsilon", type=float, default=0.00000001,
                        help="Epsilon used by optimizers")
    parser.add_argument("-w_d", "--weight_decay", type=float, default=0.0001,
                        help="Weight decay used by optimizers")
    parser.add_argument("-da", "--isDataAug", type=str, default="False",
                        help="Whether to use data augmentation or not")

    return parser.parse_args()

Overwriting argument_parser.py


### Main File

In [42]:
%%writefile main.py
from train_local import *
from argument_parser import *
import libraries

if __name__=="__main__":
  args = parse_arguments()
  trainNeuralNetwork_local(args)

Overwriting main.py


### Running

In [ ]:
# change epochs
!python3 main.py -wp DA6401_Assignment2 -we nikhithaa-iit-madras -b 32 -beta1 0.9 -beta2 0.999 -lr 0.001 -e 1 --base_dir ../Dataset/inaturalist_12K -o sgd -w_d 0  -da True

## Training sweep

In [43]:
%%writefile train_sweep.py
# import libraries
from neural_network import *
from data_loader import *
from accuracy_calculation import *
import numpy as np
import torch.nn as nn
import wandb
import gc
import os
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def trainNeuralNetwork_sweep():
  wandb.init(mode="online")
  args = wandb.config
  train_loader, test_loader, val_loader, num_classes = load_data(args["base_dir"], args["isDataAug"], args["batch_size"])
  activationFun = args["activation"]
  optimizer = args["optimizer"]
  learning_rate = args["learning_rate"]
  momentum = args["momentum"]
  beta = args["beta"]
  beta1 = args["beta1"]
  beta2 = args["beta2"]
  epsilon = args["epsilon"]
  weight_decay = args["weight_decay"]
  dropout = args["dropout"]
  num_filters = args["num_filters"]
  filter_sizes = args["filter_sizes"]
  n_neurons_denseLayer = args["n_neurons_denseLayer"]
  isBatchNormalization = args["isBatchNormalization"]
  isDataAug = args["isDataAug"]

  wandb.run.name = f"{activationFun}_{optimizer}_{dropout}_{n_neurons_denseLayer}_DataAug-{isDataAug}_BatchNorm-{isBatchNormalization}"
  best_val_accuracy = 0.0
  best_accuracy_epoch = -1

  cnn = ConvolutionalNeuralNetwork(num_classes,
                                   num_filters, filter_sizes,
                                   activationFun, optimizer,
                                   n_neurons_denseLayer,
                                   isBatchNormalization, dropout,
                                   learning_rate,
                                   momentum, beta,
                                   beta1, beta2,
                                   epsilon, weight_decay)
  cnn.to(device)

  epochs = args["epochs"]
  for epochNum in range(epochs):
    print(f"Epoch {epochNum}:")
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
      if(batch_idx % 40 == 0):
        print(f"Batch idx {batch_idx} running")
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)
      cnn.optimizer.zero_grad()
      outputs = cnn(x_batch)
      cnn.backward(outputs, y_batch)
      cnn.updateWeights()
      del x_batch, y_batch, outputs

    # Validation accuracy
    val_outputs, val_accuracy, val_loss, _, _ = findOutputs(device, cnn, val_loader)
    # wandb.run.summary["metric_name"] = val_accuracy
    print(f"validation: loss={val_loss}, accuracy={val_accuracy}")

    # Train accuracy
    train_outputs, train_accuracy, train_loss, _, _ = findOutputs(device, cnn, train_loader)
    print(f"training: loss={train_loss}, accuracy={train_accuracy}")

    if val_accuracy > best_val_accuracy:
      best_val_accuracy = val_accuracy
      best_accuracy_epoch = epochNum

    wandb.log({
        "epoch": epochNum + 1,
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy
        },commit=True)
    del val_outputs, train_outputs
    gc.collect()
    torch.cuda.empty_cache()

  wandb.log({
      "best_acc_epoch": best_accuracy_epoch,
      "best_val_accuracy": best_val_accuracy
  })

  test_outputs, test_accuracy, test_loss, test_top5_accuracy, test_top2_accuracy = findOutputs(device, cnn, test_loader, True)
  print(f"testing: loss={test_loss}, top1_accuracy={test_accuracy}, top5_accuracy = {test_top5_accuracy}, top2_accuracy = {test_top2_accuracy}")

  wandb.log({
      "test_loss": test_loss,
      "test_top1_accuracy": test_accuracy,
      "test_top5_accuracy": test_top5_accuracy,
      "test_top2_accuracy": test_top2_accuracy
  })
  del cnn,train_loader, test_loader, val_loader
  gc.collect()
  torch.cuda.empty_cache()

  wandb.finish()

Overwriting train_sweep.py


###  Main Sweep

In [44]:
%%writefile main_sweep.py
from train_sweep import *
# import libraries
import wandb

sweep_configuration = {
    "method": "random",
    "name" : "finetune_final_sweep",
    "parameters": {
        "num_filters": {'values': [[32, 32, 32, 32, 32]]},  # have no effect since using pre-trained model
        "filter_sizes": {'values': [[3, 3, 3, 3, 3]]},        # have no effect since using pre-trained model
        "activation": {"values": ["ReLU"]},   # have no effect since using pre-trained model
        "optimizer": {"values": ["adam", "rmsprop", "sgd"]},
        "learning_rate": {"values": [1e-3, 1e-4]},
        "weight_decay": {"values": [0.0001, 0.0005]},
        "momentum": {"values": [0.9]},
        "beta": {"values": [0.9]},
        "beta1": {"values":[0.9]},
        "beta2": {"values": [0.999]},
        "epsilon": {"values": [1e-8]},
        # "base_dir": {"values":["/content/drive/MyDrive/DL_Assignment2/Dataset/inaturalist_12K/"]},
        "base_dir": {"values": ["../Dataset/inaturalist_12K"]},
        "isDataAug": {"values": ["False", "True"]},
        "isBatchNormalization": {"values": ["False"]},  # have no effect since using pre-trained model
        "dropout": {"values": [0]},            # have no effect since using pre-trained model
        "n_neurons_denseLayer": {"values": [128]},   # have no effect since using pre-trained model
        "batch_size": {"values": [32,64]},
        "epochs": {"values": [5,10]}
    }
}

if __name__=="__main__":
  wandb.login()
  wandb_id = wandb.sweep(sweep_configuration, project="DA6401_Assignment2")
  wandb.agent(wandb_id, function=trainNeuralNetwork_sweep)

Overwriting main_sweep.py


### Running

In [ ]:
!python3 main_sweep.py

In [ ]:
# wandb.login(key="x")
# wandb_id = wandb.sweep(sweep_configuration, project="DA6401_Assignment2")
# wandb.agent(wandb_id, function=trainNeuralNetwork_sweep)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nikhithaa (nikhithaa-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Create sweep with ID: 0kqsutes
Sweep URL: https://wandb.ai/nikhithaa-iit-madras/DA6401_Assignment2/sweeps/0kqsutes


wandb: Agent Starting Run: 3ypawy7j with config:
wandb: 	activation: ReLU
wandb: 	base_dir: /kaggle/input/inaturalist/inaturalist_12K
wandb: 	batch_size: 32
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	dropout: 0
wandb: 	epochs: 10
wandb: 	epsilon: 1e-08
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	isBatchNormalization: False
wandb: 	isDataAug: False
wandb: 	learning_rate: 0.0001
wandb: 	momentum: 0.9
wandb: 	n_neurons_denseLayer: 128
wandb: 	num_filters: [32, 32, 32, 32, 32]
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.0005
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 178MB/s] 


Epoch 0:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
Batch idx 160 running
Batch idx 200 running
Batch idx 240 running
validation: loss=1.6043938837051392, accuracy=74.85
training: loss=1.6057432772353495, accuracy=74.34679334916865
Epoch 1:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
Batch idx 160 running
Batch idx 200 running
Batch idx 240 running
validation: loss=1.1927816009521484, accuracy=79.25
training: loss=1.2003781593714764, accuracy=78.08476059507439
Epoch 2:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
Batch idx 160 running
Batch idx 200 running
Batch idx 240 running
validation: loss=0.9676698985099792, accuracy=81.05
training: loss=0.9807530491631364, accuracy=79.73496687085886
Epoch 3:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
Batch idx 160 running
Batch idx 200 running
Batch idx 240 running
validation: loss=0.834

...

best_acc_epoch,▁
best_val_accuracy,▁
epoch,▁▃▅▆█
test_loss,▁
test_top1_accuracy,▁
test_top2_accuracy,▁
test_top5_accuracy,▁
train_accuracy,▁▄▆▇█
train_loss,█▄▂▂▁
val_accuracy,▁▆█▆█
val_loss,█▄▂▂▁


wandb: Agent Starting Run: qzj6pcia with config:
wandb: 	activation: ReLU
wandb: 	base_dir: /kaggle/input/inaturalist/inaturalist_12K
wandb: 	batch_size: 64
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	dropout: 0
wandb: 	epochs: 10
wandb: 	epsilon: 1e-08
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	isBatchNormalization: False
wandb: 	isDataAug: False
wandb: 	learning_rate: 0.0001
wandb: 	momentum: 0.9
wandb: 	n_neurons_denseLayer: 128
wandb: 	num_filters: [32, 32, 32, 32, 32]
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.0001


Epoch 0:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.216280937194824, accuracy=24.3
training: loss=2.2157239029893043, accuracy=23.577947243405426
Epoch 1:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.1112672119140625, accuracy=41.6
training: loss=2.1050654601299668, accuracy=42.367795974496815
Epoch 2:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.003027765274048, accuracy=55.8
training: loss=2.002930264008583, accuracy=55.081885235654454
Epoch 3:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.9110452480316162, accuracy=63.85
training: loss=1.9093199649413297, accuracy=62.40780097512189
Epoch 4:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.8234469785690308, accuracy=68.0
training: loss=1.82534673969840

wandb: Ctrl + C detected. Stopping sweep.


epoch,▁▂▄▅▇█
train_accuracy,▁▄▆▇██
train_loss,█▆▅▃▂▁
val_accuracy,▁▄▆▇▇█
val_loss,█▆▅▃▂▁
epoch,6
train_accuracy,69.87123
train_loss,1.74669
val_accuracy,71.45
val_loss,1.74328
